In [2]:
import sys, os #imp
sys.path.append('..')
import pandas as pd
import yaml
import backtesting as bt
import matplotlib.pyplot as plt
import numpy as np
from tools.instruments import instruments
from tools.display import highchart

In [3]:
%load_ext autoreload
%aimport backtesting.system
%aimport backtesting.trader
%aimport tools.quotes
%aimport tools.instruments
%aimport backtesting.trades
%aimport tools.display
%autoreload 2

In [34]:
with open('systems/sample.yaml', encoding='utf-8') as f:
    system1 = yaml.load(f, Loader=yaml.FullLoader)
    print(system1)

{'name': 'testSystem', 'description': '시스템 설명', 'sectors': None, 'instruments': ['AD', 'C', 'W', 'NQ'], 'from_date': '2021-01-01', 'to_date': None, 'principal': 100000, 'heat_system': 'DefaultHeat', 'max_system_heat': 0.3, 'max_sector_heat': 0.3, 'max_trade_heat': 0.3, 'max_lots': 1, 'commission': 5.98, 'skid': 20, 'allow_pyramiding': False, 'metrics': [['trendup', 'TREND', 'period=50', "direction='up'", 'threshold=0.7'], ['min50', 'MIN', 'period=50']], 'entry_rule': {'long': 'trendup>3', 'short': None}, 'exit_rule': {'long': 'low<min50', 'short': None}, 'stop_rule': {'long': 'min50', 'short': None}}


In [35]:
trader = bt.Trader(systems=[system1])
system = trader.systems[0]
trader.run()

매매시작
거래일: 2021-01-04 00:00:00, 시스템: testSystem
거래일: 2021-01-05 00:00:00, 시스템: testSystem
거래일: 2021-01-06 00:00:00, 시스템: testSystem
거래일: 2021-01-07 00:00:00, 시스템: testSystem
거래일: 2021-01-08 00:00:00, 시스템: testSystem
거래일: 2021-01-11 00:00:00, 시스템: testSystem
거래일: 2021-01-12 00:00:00, 시스템: testSystem
거래일: 2021-01-13 00:00:00, 시스템: testSystem
거래일: 2021-01-14 00:00:00, 시스템: testSystem
거래일: 2021-01-15 00:00:00, 시스템: testSystem
거래일: 2021-01-18 00:00:00, 시스템: testSystem
거래일: 2021-01-19 00:00:00, 시스템: testSystem
거래일: 2021-01-20 00:00:00, 시스템: testSystem
거래일: 2021-01-21 00:00:00, 시스템: testSystem
거래일: 2021-01-22 00:00:00, 시스템: testSystem
거래일: 2021-01-25 00:00:00, 시스템: testSystem
거래일: 2021-01-26 00:00:00, 시스템: testSystem
거래일: 2021-01-27 00:00:00, 시스템: testSystem
거래일: 2021-01-28 00:00:00, 시스템: testSystem
거래일: 2021-01-29 00:00:00, 시스템: testSystem
거래일: 2021-02-01 00:00:00, 시스템: testSystem
거래일: 2021-02-02 00:00:00, 시스템: testSystem
거래일: 2021-02-03 00:00:00, 시스템: testSystem
거래일: 2021-02-04 00:00:00, 시스템

거래일: 2022-04-04 00:00:00, 시스템: testSystem
거래일: 2022-04-05 00:00:00, 시스템: testSystem
거래일: 2022-04-06 00:00:00, 시스템: testSystem
거래일: 2022-04-07 00:00:00, 시스템: testSystem
거래일: 2022-04-08 00:00:00, 시스템: testSystem
거래일: 2022-04-11 00:00:00, 시스템: testSystem
거래일: 2022-04-12 00:00:00, 시스템: testSystem
거래일: 2022-04-13 00:00:00, 시스템: testSystem
거래일: 2022-04-14 00:00:00, 시스템: testSystem
거래일: 2022-04-18 00:00:00, 시스템: testSystem
거래일: 2022-04-19 00:00:00, 시스템: testSystem
거래일: 2022-04-20 00:00:00, 시스템: testSystem
거래일: 2022-04-21 00:00:00, 시스템: testSystem
거래일: 2022-04-22 00:00:00, 시스템: testSystem
거래일: 2022-04-25 00:00:00, 시스템: testSystem
거래일: 2022-04-26 00:00:00, 시스템: testSystem
거래일: 2022-04-27 00:00:00, 시스템: testSystem
거래일: 2022-04-28 00:00:00, 시스템: testSystem
거래일: 2022-04-29 00:00:00, 시스템: testSystem
거래일: 2022-05-02 00:00:00, 시스템: testSystem
거래일: 2022-05-03 00:00:00, 시스템: testSystem
거래일: 2022-05-04 00:00:00, 시스템: testSystem
매매종료


In [36]:
system.create_report()

In [16]:
trades = system.trades.get()

In [24]:
trades[0].exittype

'STOP'

In [17]:
trades[0].result

'WIN'

In [100]:
instruments['W'].quotes().TREND(direction='up',period=50,threshold=0.5)

1999-03-16    0
1999-03-17    0
1999-03-18    0
1999-03-19    0
1999-03-22    0
             ..
2022-04-28    0
2022-04-29    0
2022-05-02    0
2022-05-03    0
2022-05-04    0
Name: trendup50, Length: 5833, dtype: int32

In [5]:
instruments.sectors(instruments.db_symbols())

{'currency': ['AD', 'BP', 'CD', 'DX', 'EC', 'JY', 'MP', 'NE', 'SF'],
 'grain': ['BO', 'C', 'KW', 'O', 'S', 'SM', 'W'],
 'tropical': ['CC', 'KC', 'OJ', 'SB'],
 'petroleum': ['CL', 'G', 'HO', 'NG', 'RB'],
 'fiber': ['CT'],
 'interest-rate': ['ED', 'FF', 'FV', 'TU', 'TY', 'US'],
 'equity': ['ES', 'MD', 'NK', 'NQ', 'RTY', 'SP'],
 'metal': ['GC', 'HG', 'PA', 'PL', 'SI'],
 'meat': ['LC', 'LN']}

In [1]:
from itertools import groupby

In [5]:
data =[0,0,0,1,0,1,1]

In [8]:
[np.array(v).cumsum() for k,v in groupby(data)]

[array([<itertools._grouper object at 0x0000024ED8C6DAF0>], dtype=object),
 array([<itertools._grouper object at 0x0000024ED8C6D0D0>], dtype=object),
 array([<itertools._grouper object at 0x0000024ED8C6DE20>], dtype=object),
 array([<itertools._grouper object at 0x0000024ED8C6D4C0>], dtype=object)]

In [9]:
v

NameError: name 'v' is not defined

In [61]:
metrics

[['uptrend', 'TREND', "direction='up'", 'period=50'],
 ['downtrend', 'TREND', "direction='down'", 'period=50'],
 ['min30', 'MIN', 'period=50']]

In [67]:
d = {}
for metric in metrics:
    d[metric[0]] = [k for k,v in axes.items() if metric[1] in v][0]

In [68]:
d

{'uptrend': 'trend', 'downtrend': 'trend', 'min30': 'ohlc'}

In [3]:
s = (df.values == data)

NameError: name 'df' is not defined